# Fix pathing

In [1]:
import sys


sys.path.append("..")


In [2]:
import constants

import os


constants.PROJECT_DIRECTORY_PATH = os.path.dirname(os.path.dirname(constants.PROJECT_DIRECTORY_PATH))


# Imports

In [3]:
import datahandler
import utils
import pathing

import osmnx as ox
import networkx
import random
import time
from tqdm import tqdm


# Constants

In [4]:
data_preprocessor = datahandler.DataPreprocessorOUS_V2()
data_preprocessor.execute()

data_loader = datahandler.DataLoader(datahandler.DataPreprocessorOUS_V2)
data_loader.execute(clean=False, processed=False, enhanced=False)

Enhancing dataset: 100%|██████████| 2/2 [00:00<?, ?it/s]


# Methods

In [5]:
def get_node(od: pathing.OriginDestination, x, y):
    if (x, y) in od.node_cache:
        return od.node_cache[(x, y)]
    
    while True:
        node = ox.distance.nearest_nodes(od.graph, x, y)
        if networkx.has_path(od.graph, node, od.node_validator) and networkx.has_path(od.graph, od.node_validator, node):
            od.node_cache[(x, y)] = node
            return node
        else:
            od.graph.remove_node(node)


In [6]:
def evaluate_travel_time_accuracy(benchmark_times, calculated_times):
    total_diff = 0
    total_diff_percentage = 0

    for destination, benchmark in benchmark_times.items():
        if destination in calculated_times:
            calculated = calculated_times[destination]
            diff = abs(benchmark - calculated)
            diff_percentage = (diff / benchmark) * 100

            total_diff += diff
            total_diff_percentage += diff_percentage

            print(f"Destination {destination}:")
            print(f"  Benchmark Time: {benchmark} minutes")
            print(f"  Calculated Time: {calculated} minutes")
            print(f"  Difference: {diff} minutes ({diff_percentage:.2f}%)")
            print()
        else:
            print(f"Destination {destination} not found in calculated times.")

    avg_diff_percentage = total_diff_percentage / len(benchmark_times)
    print(f"Total Discrepancy: {total_diff} minutes across all destinations.")
    print(f"Average Discrepancy Percentage: {avg_diff_percentage:.2f}%")


In [7]:
def calculate_error(od: pathing.OriginDestination, benchmark_times, start_node):
    calculated_times = {}
    
    # Assuming utils and get_node functions are defined and work as intended
    for (lon, lat), _ in benchmark_times.items():
        # Convert geographic coordinates to UTM and get the corresponding node
        x, y = utils.geographic_to_utm(lon, lat)
        destination_node = get_node(od, x, y)

        # Calculate the shortest path and total travel time
        shortest_time_path = ox.shortest_path(
            od.graph,
            start_node,
            destination_node,
            weight='time'
        )
        total_travel_time = sum(od.graph[u][v][0]['time'] for u, v in zip(shortest_time_path[:-1], shortest_time_path[1:]))

        # Store the calculated travel time
        calculated_times[(lon, lat)] = total_travel_time

    # Calculate MSE
    errors = [abs(benchmark_times[key] - calculated_times[key]) for key in benchmark_times]

    mse = sum(error ** 2 for error in errors) / len(errors)
    
    return mse


In [8]:
def gen_linkers(road_type_factors, link_factor):
    linkables = [
        "motorway",
        "trunk",
        "primary",
        "secondary",
        "tertiary",
    ]

    for linkable in linkables:
        road_type_factors[linkable + "_link"] = road_type_factors[linkable] * link_factor
    
    return road_type_factors


In [18]:
def optimize_parameters(od: pathing.OriginDestination, benchmark_times, start_node, max_duration_minutes):
    start_time = time.time()
    max_duration_seconds = max_duration_minutes * 60
    
    # Define parameter bounds
    param_bounds = {
        "default_intersection_penalty": (5, 20),
        "traffic_signal_penalty": (5, 30),
        "linkable_factor": (0.75, 1.0),
        "road_type_factors": {
            "motorway": (0.8, 1.0),
            "trunk": (0.8, 1.0),
            "primary": (0.7, 0.9),
            "secondary": (0.7, 0.9),
            "tertiary": (0.7, 0.9),
            "unclassified": (0.6, 0.8),
            "residential": (0.5, 0.7),
            "living_street": (0.3, 0.5)
        }
    }
    
    # Initialize the best solution
    best_solution = None
    best_error = float("inf")
    
    # Progress bar
    with tqdm(desc="Optimizing Parameters") as pbar:
        while time.time() - start_time < max_duration_seconds:
            # Generate a random solution within the bounds
            current_solution = {
                "default_intersection_penalty": random.randint(*param_bounds["default_intersection_penalty"]),
                "traffic_signal_penalty": random.randint(*param_bounds["traffic_signal_penalty"]),
                "road_type_factors": {k: random.uniform(*v) for k, v in param_bounds["road_type_factors"].items()}
            }
            linkable_factor = random.uniform(*param_bounds["linkable_factor"])

            current_solution["road_type_factors"] = gen_linkers(current_solution["road_type_factors"], linkable_factor)
            
            # Set graph weights with the current solution
            od.set_graph_weights_v2(**current_solution)
            
            # Calculate the error for the current solution
            current_error = calculate_error(od, benchmark_times, start_node)
            
            # Update best solution if the current one is better
            if current_error < best_error:
                best_solution = current_solution
                best_linkable_factor = linkable_factor
                best_error = current_error
                tqdm.write(
                    f"    New best solution:\n"
                    f"        Default Intersection Penalty: {best_solution['default_intersection_penalty']}\n"
                    f"        Traffic Signal Penalty: {best_solution['traffic_signal_penalty']}\n"
                    f"        Linkable Factor: {best_linkable_factor}\n"
                    f"        Road Type Factors:\n"
                    + "\n".join([f"            {k}: {v}" for k, v in best_solution['road_type_factors'].items()]) +
                    f"\n        Error: {best_error}"
                )
            
            # Update progress bar
            pbar.update()
    
    return best_solution, best_error


# Main

In [10]:
od = pathing.OriginDestination(
    dataset_id="oslo",
    utm_epsg=f"EPSG:326{33}"
)

od.get_graph()

central_depot_grid_id = 22620006649000
central_depot_x, central_depot_y = utils.id_to_utm(central_depot_grid_id)
od.node_validator = ox.distance.nearest_nodes(od.graph, central_depot_x, central_depot_y)


In [11]:
x, y = utils.geographic_to_utm(10.7343661, 59.9369806)
start_node = get_node(od, x, y)

benchmark_times = {
    (10.6572104, 59.6800045): 40,
    (10.6661424, 59.8598299): 60,
    (10.4852080, 59.8073283): 35,
    (11.8086592, 60.2271721): 68,
    (10.7193651, 60.1973847): 53,
    (10.7622022, 60.0035569): 14
}


In [19]:
best_solution, best_error = optimize_parameters(
    od,
    benchmark_times,
    start_node,
    max_duration_minutes=10
)


Optimizing Parameters: 0it [00:00, ?it/s]

Optimizing Parameters: 1it [00:03,  3.09s/it]

    New best solution:
        Default Intersection Penalty: 14
        Traffic Signal Penalty: 10
        Linkable Factor: 0.7828180563961024
        Road Type Factors:
            motorway: 0.9631405233226625
            trunk: 0.9123215912448058
            primary: 0.7256570752436865
            secondary: 0.7297436395842564
            tertiary: 0.798232035660831
            unclassified: 0.7481793772088875
            residential: 0.5248171368985575
            living_street: 0.3495023411373888
            motorway_link: 0.7539637925037715
            trunk_link: 0.7141818148664583
            primary_link: 0.5680574612523429
            secondary_link: 0.5712564976067654
            tertiary_link: 0.624870450709116
        With error: 53.00167901988586


Optimizing Parameters: 3it [00:09,  3.17s/it]

    New best solution:
        Default Intersection Penalty: 10
        Traffic Signal Penalty: 20
        Linkable Factor: 0.886599217114875
        Road Type Factors:
            motorway: 0.8518126330959961
            trunk: 0.9822347695905322
            primary: 0.8707746243853463
            secondary: 0.792667278441288
            tertiary: 0.7058460232625422
            unclassified: 0.7488420854582375
            residential: 0.5118301224394534
            living_street: 0.33203758868203803
            motorway_link: 0.7552164136314704
            trunk_link: 0.8708485777419755
            primary_link: 0.7720281002635473
            secondary_link: 0.7027781884986246
            tertiary_link: 0.6258025316282178
        With error: 23.125947951786618


...

Optimizing Parameters: 93it [05:02,  3.25s/it]

    New best solution:
        Default Intersection Penalty: 7
        Traffic Signal Penalty: 22
        Linkable Factor: 0.8342989094156716
        Road Type Factors:
            motorway: 0.9316073724923927
            trunk: 0.9699265632580953
            primary: 0.8721456891098264
            secondary: 0.7743084386462568
            tertiary: 0.8408122082647067
            unclassified: 0.7777146530071191
            residential: 0.6142154053141573
            living_street: 0.44117352214177397
            motorway_link: 0.7772390148740026
            trunk_link: 0.8092086739395193
            primary_link: 0.7276301972759076
            secondary_link: 0.6460046859139235
            tertiary_link: 0.7014887083786274
        With error: 13.943516996505636


Optimizing Parameters: 98it [05:20,  3.53s/it]

    New best solution:
        Default Intersection Penalty: 7
        Traffic Signal Penalty: 23
        Linkable Factor: 0.9367932723284527
        Road Type Factors:
            motorway: 0.9829123702090651
            trunk: 0.9890896028703644
            primary: 0.8022311450894829
            secondary: 0.8391160216994574
            tertiary: 0.7814750888642189
            unclassified: 0.610927540515509
            residential: 0.6595426202076038
            living_street: 0.34674379908080605
            motorway_link: 0.9207856957002656
            trunk_link: 0.9265724856989784
            primary_link: 0.7515247395721784
            secondary_link: 0.7860782438310676
            tertiary_link: 0.73208060574028
        With error: 12.296442323521227


Optimizing Parameters: 180it [10:03,  3.35s/it]


In [20]:
od.set_graph_weights_v2(**best_solution)

calculated_times = {}

for lon, lat in benchmark_times.keys():
    # gets destination node for UTM coordinates
    x, y = utils.geographic_to_utm(lon, lat)
    destination_node = get_node(od, x, y)

    # get path of nodes from start node to destination node
    shortest_time_path = ox.shortest_path(
        od.graph,
        start_node,
        destination_node,
        weight='time'
    )

    # caclulate travel time between start and destination node and cache the result
    total_travel_time = sum(od.graph[u][v][0]['time'] for u, v in zip(shortest_time_path[:-1], shortest_time_path[1:]))
    calculated_times[(lon, lat)] = total_travel_time
    
evaluate_travel_time_accuracy(benchmark_times, calculated_times)


Destination (10.6572104, 59.6800045):
  Benchmark Time: 40 minutes
  Calculated Time: 40.3363019423618 minutes
  Difference: 0.3363019423617999 minutes (0.84%)

Destination (10.6661424, 59.8598299):
  Benchmark Time: 60 minutes
  Calculated Time: 58.64136638922261 minutes
  Difference: 1.3586336107773889 minutes (2.26%)

Destination (10.485208, 59.8073283):
  Benchmark Time: 35 minutes
  Calculated Time: 35.66103216792497 minutes
  Difference: 0.6610321679249722 minutes (1.89%)

Destination (11.8086592, 60.2271721):
  Benchmark Time: 68 minutes
  Calculated Time: 75.14389705743733 minutes
  Difference: 7.143897057437329 minutes (10.51%)

Destination (10.7193651, 60.1973847):
  Benchmark Time: 53 minutes
  Calculated Time: 48.49561208821867 minutes
  Difference: 4.504387911781329 minutes (8.50%)

Destination (10.7622022, 60.0035569):
  Benchmark Time: 14 minutes
  Calculated Time: 14.240687561502401 minutes
  Difference: 0.24068756150240134 minutes (1.72%)

Total Discrepancy: 14.2449402